1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
*название источника,
*наименование новости,
*ссылку на новость,
*дата публикации

2)Сложить все новости в БД

In [1]:
from pymongo import MongoClient
import requests
from pprint import pprint
from lxml import html
import pandas as pd
import re
import datetime
import json

In [2]:
def request_to_lenta():
    result = pd.DataFrame()
    header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    main_link = 'https://www.lenta.ru/'
    main_l = 'https://lenta.ru'
    try:
        response = requests.get(main_link, headers = header)
    except:
        print('Ошибка запроса')
    root = html.fromstring(response.text)
    news = root.xpath("//section[contains(@class,'b-top7-for-main')]//time[@class='g-time']")
    names = []
    links = []
    date = []
    from_n = []
    for i in news:
        name = i.xpath("./..//text()")[1]
        link = i.xpath("./..//@href")[0]
        flag = re.search(r'^/', link)
        if flag:
            link = f'{main_l}{link}'
        date_one = i.xpath(".//@datetime")[0]
        names.append(name)
        links.append(link)
        date.append(date_one)
    for i in links:
        try:
            response_from = requests.get(i, headers = header)
            root_from = html.fromstring(response_from.text)
            from_news = root_from.xpath("//section[contains(@class, 'b-links')]/div[2]//a/@href")[0]
            from_n.append(from_news)
        except:
            from_n.append(f'ошибка в получении кода по ссылке {i}')
    result['names'] = names
    result['links'] = links
    result['date'] = date
    result['from'] = from_n 
    return result



result = request_to_lenta()
result

,names,links,date,from
0,Турция снова нанесла удар по сирийской армии,https://lenta.ru/news/2020/02/11/turkish/,"19:49, 11 февраля 2020",https://lenta.ru/news/2020/02/11/erd/
1,В России пожалели о попытке Польши превратить ...,https://lenta.ru/news/2020/02/11/posolstvo/,"20:50, 11 февраля 2020",https://lenta.ru/news/2020/01/21/skajinemolchi/
2,Пассажиры охваченного смертельным вирусом лайн...,https://lenta.ru/news/2020/02/11/whynotwine/,"20:37, 11 февраля 2020",https://lenta.ru/news/2020/02/11/freeporn/
3,Смертельный китайский вирус ударил по российск...,https://lenta.ru/news/2020/02/11/aviafails/,"20:33, 11 февраля 2020",https://www.rbc.ru/business/11/02/2020/5e4264a...
4,Память умерших почтили сотнями пустых бутылок,https://lenta.ru/news/2020/02/11/bottlememorial/,"20:24, 11 февраля 2020",https://www.fox2detroit.com/news/family-honors...
5,Раскрыта причина увольнения главы офиса Зеленс...,https://lenta.ru/news/2020/02/11/vatikan/,"20:20, 11 февраля 2020",https://www.pravda.com.ua/rus/articles/2020/02...
6,Умер чемпион СССР в составе «Спартака» Валерий...,https://lenta.ru/news/2020/02/11/valery/,"20:17, 11 февраля 2020",https://rsport.ria.ru/20200211/1564568467.html
7,Киселев заступился за посмеявшуюся над льготам...,https://lenta.ru/news/2020/02/11/kiselev/,"20:13, 11 февраля 2020",https://info24.ru/news/kiselev-otreagiroval-na...
8,Кожа начала слезать с лица женщины после приме...,https://lenta.ru/news/2020/02/11/skinpeelingoff/,"20:09, 11 февраля 2020",https://www.thesun.co.uk/fabulous/10940669/pou...
9,Домашний пес помог семье найти утечку угарного...,https://lenta.ru/news/2020/02/11/leak/,"20:05, 11 февраля 2020",https://www.fox17online.com/news/local-news/la...


In [3]:
def request_to_ya():
    result = pd.DataFrame()
    header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    main_link = 'https://yandex.ru/news/'
    main_l = 'https://yandex.ru'
    now = datetime.datetime.now()
    try:
        response = requests.get(main_link, headers = header)
    except:
        print('Ошибка запроса')
    root = html.fromstring(response.text)
    news = root.xpath("//h2")
    names = []
    links = []
    date = []
    from_n = []
    for i in news:
        name = i.xpath(".//text()")[0]
        link = i.xpath(".//@href")[0]
        flag = re.search(r'^/', link)
        if flag:
            link = f'{main_l}{link}'
        names.append(name)
        links.append(link)
    
    news_two = root.xpath("//div[@class='story__info']")
    for i in news_two:
        date_one = i.xpath(".//text()")[0]
        from_news = re.sub(r'\d\d:\d\d', '', date_one)
        date_one = re.search(r'\d\d:\d\d', date_one)[0]
        date.append(f'{date_one}, {now.strftime("%d %B %Y")}')
        from_n.append(from_news)
    result['names'] = names
    result['links'] = links
    result['date'] = date
    result['from'] = from_n 
    return result


result_two = request_to_ya()
print(len(result_two))
result_two.tail(5)

65


,names,links,date,from
60,Kia опубликовала изображения нового Kia Sorento,https://yandex.ru/news/story/Kia_opublikovala_...,"20:24, 11 February 2020",SpeedMe
61,Автомобили LADA лидируют на вторичном рынке РФ,https://yandex.ru/news/story/Avtomobili_LADA_l...,"18:36, 11 February 2020",32CARS.ru
62,КамАЗ запатентовал электрогрузовик без кабины,https://yandex.ru/news/story/KamAZ_zapatentova...,"20:12, 11 February 2020",МИР 24
63,Nissan может приостановить выпуск автомобилей ...,https://yandex.ru/news/story/Nissan_mozhet_pri...,"18:00, 11 February 2020",32CARS.ru
64,В России отзывают Renault Logan и Sandero,https://yandex.ru/news/story/V_Rossii_otzyvayu...,"19:09, 11 February 2020",За рулем Кубань


In [4]:
def request_to_mail():
    result = pd.DataFrame()
    header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    main_link = 'https://news.mail.ru/'
    main_l = 'https://news.mail.ru'
    now = datetime.datetime.now()
    try:
        response = requests.get(main_link, headers = header)
    except:
        print('Ошибка запроса')
    root = html.fromstring(response.text)
    news = root.xpath("//div[contains(@class,'cols__column_small_percent-50')]/*/*[contains(@class,'list')]|//div[contains(@class,'cols__column_small_percent-50')]/*/*[contains(@class,'newsitem')]//a")
    names = []
    links = []
    date = []
    from_n = []
    for i in news:
        name = i.xpath(".//text()")
        for j in name:
            names.append(j)
        link = i.xpath(".//@href")
        for k in link:
            flag = re.search(r'^/', k)
            if flag:
                k = f'{main_l}{k}'
            links.append(k)
    for i in links:
        try:
            response_from = requests.get(i, headers = header)
            root_from = html.fromstring(response_from.text)
            date_news = root_from.xpath("//span[@class='note__text breadcrumbs__text js-ago']//@datetime")[0]
            date_news = re.sub(r'T', ' ', date_news)
            date_news = re.sub(r'\+03:00', '', date_news)
            date.append(date_news)
            from_news = root_from.xpath("//a[@class='link color_gray breadcrumbs__link']//text()")[0]
            from_n.append(from_news)
        except:
            from_n.append(f'ошибка в получении кода по ссылке {i}')
    result['names'] = names
    result['links'] = links
    result['date'] = date
    result['from'] = from_n 
    return result


result_three = request_to_mail()
result_three.tail()

,names,links,date,from
17,Пользователи Сети высмеяли невесту за неудачно...,https://news.mail.ru/society/40544209/,2020-02-11 15:52:45,Новости Mail.ru
18,За 2019 год российские суды прекратили более п...,https://news.mail.ru/incident/40542514/,2020-02-11 18:10:36,Новый Калининград
19,Источник сообщил о гибели пилота сбитого в Идл...,https://news.mail.ru/incident/40544369/,2020-02-11 16:04:36,РИА Новости
20,"В Тюмени выписана студентка, заразившаяся коро...",https://news.mail.ru/incident/40543317/,2020-02-11 15:03:37,Интерфакс
21,СМИ: боевики сирийской оппозиции сбили вертоле...,https://news.mail.ru/incident/40541919/,2020-02-11 14:14:42,ТАСС


In [5]:
client  = MongoClient('localhost',27017)
db = client['news']
news_lenta = db.news.news_lenta
news_ya = db.news.news_ya
news_mail = db.news.news_mail
news = db.news

In [6]:
def load_mongo(page, name_mongo):
    j_page = page.to_json(orient='records')
    j_dict = json.loads(j_page)
    for i in j_dict:
        i['_id'] = i['links']
    try:
        name_mongo.insert_many(j_dict)
        print('данные добавлены')
    except:
        print('часть данных дублируется, используйте load_mongo_update')

In [7]:
load_mongo(result, news_lenta)
load_mongo(result_two, news_ya)
load_mongo(result_three, news_mail)

часть данных дублируется, используйте load_mongo_update
часть данных дублируется, используйте load_mongo_update
часть данных дублируется, используйте load_mongo_update


In [8]:
def load_mongo_update(page, name_mongo):
    choose = input('Обновлять старые вакансии? y/n ')
    for i in range(len(page)):
        link = page.loc[i, 'links']
        all_sal = name_mongo.count_documents({'_id':{'$eq':link}})
        if choose == 'y':
            j_page = page.loc[i].to_json(orient='index')
            j_dict = json.loads(j_page)
            name_mongo.update_one({'links':link}, {'$set':j_dict})
            if all_sal is 0:
                print(link, 'new')
        elif (all_sal is 0) and (choose == 'n'):
            j_page = page.loc[i].to_json(orient='index')
            j_dict = json.loads(j_page)
            j_dict['_id'] = link
            name_mongo.insert_one(j_dict)
            print(link, 'new')

In [9]:
load_mongo_update(result, news_lenta)
load_mongo_update(result_two, news_ya)
load_mongo_update(result_three, news_mail)

Обновлять старые вакансии? y/n y
https://lenta.ru/news/2020/02/11/posolstvo/ new
Обновлять старые вакансии? y/n y
https://yandex.ru/news/story/EHks-prezidentam_RF_predlagayut_razreshit_pozhiznenno_byt_senatorami--b7311b6fde908133868beb0fdd0c0e87?lr=213&lang=ru&stid=RMiHUcTLA0Sya-4cjI2a&persistent_id=88034792&rubric=index&from=index new
https://yandex.ru/sport/story/Veteran_Spartaka_Rejngold_skonchalsya--0fdf8ee5a090cb81a397f32d4d808e56?lr=213&lang=ru&stid=TEPZAzm5SngDuHNEjGOf&persistent_id=88061767&rubric=index&from=index new
https://yandex.ru/news/story/Gorodskie_subbotniki_v_Moskve_projdut_11_i_25_aprelya--dc27b1989aac828efbfe8e1b4d0748b1?lr=213&lang=ru&stid=_Alw7lsZ4uXx75wNAhim&persistent_id=88044435&rubric=Moscow&from=index new
https://yandex.ru/news/story/Den_vsekh_vlyublennykh_otmetyat_v_kulturnykh_centrakh_i_bibliotekakh_Moskvy--f5a5fd572f3d5beda03b4ab6df09a12a?lr=213&lang=ru&stid=ix007TF_mibvxT5uceIB&persistent_id=87918761&rubric=Moscow&from=index new
https://yandex.ru/news/sto

Обновлять старые вакансии? y/n y
https://news.mail.ru/economics/40534635/ new
https://news.mail.ru/society/40547282/ new
